## **DATASET**

In [ ]:
import pandas as pd

df = pd.read_csv("RTA Dataset.csv")
df

## **Seleção de Features**

In [ ]:
#features = ['Age_band_of_driver', 'Number_of_vehicles_involved', 'Number_of_casualties', 'Driving_experience', 'Type_of_collision', 'Type_of_vehicle', 'Area_accident_occured']

#features = ['Age_band_of_driver', 'Number_of_vehicles_involved', 'Number_of_casualties', 'Driving_experience', 'Type_of_collision']

features = ['Age_band_of_driver', 'Number_of_vehicles_involved', 'Number_of_casualties', 'Driving_experience']

## **Tratamento do DataSet**

**Transformação de Valores Categóricos em Valores Numéricos**

In [ ]:
#Converter 'Driving_experience' para valores numéricos
experiencia = {
    'Less than 1yr': 1,
    '1-2yr': 2,
    '2-5yr': 3,
    '5-10yr': 4,
    'Above 10yr': 5
}

df['Driving_experience'] = df['Driving_experience'].map(experiencia)

#Converter 'Age_band_of_driver' para valores numéricos 
idade = {
    'Under 18': 1,
    '18-30': 2,
    '31-50': 3,
    'Over 51': 4,
    'Unknown': 5
}

df['Age_band_of_driver'] = df['Age_band_of_driver'].map(idade)

#Converter 'Type_of_vehicle' para valores numéricos
veiculo = {
    'Automobile': 1,
    'Public (> 45 seats)': 2,
    'Lorry (41?100Q)': 3,
    'Vehicle with vehicle collision': 4,
    'Public (13?45 seats)': 5,
    'Lorry (11?40Q)': 6,
    'Long lorry': 7,
    'Public (12 seats)': 8,
    'Taxi': 9,
    'Pick up upto 10Q': 10,
    'Stationwagen': 11,
    'Ridden horse': 12,
    'Other': 13,
    'Bajaj': 14,
    'Turbo': 15,
    'Motorcycle': 16,
    'Special vehicle': 17,
    'Bicycle': 18
}

df['Type_of_vehicle'] = df['Type_of_vehicle'].map(veiculo)

#Converter 'Type_of_collision' para valores numéricos
colisao = {
    'Collision with roadside-parked vehicles': 1,
    'Vehicle with vehicle collision': 2,
    'Collision with roadside objects': 3,
    'Collision with animals': 4,
    'Other': 5,
    'Rollover': 6,
    'Fall from vehicles': 7,
    'Collision with pedestrians': 8,
    'With Train': 9,
    'Unknown': 10
}

df['Type_of_collision'] = df['Type_of_collision'].map(colisao)

#Converter 'Area_accident_occured' para valores numéricos
area = {
    'Residential areas': 1,
    'Office areas': 2,
    'Recreational areas': 3,
    'Industrial areas': 4,
    'Other': 5,
    'Church areas': 6,
    'Market areas': 7,
    'Unknown': 8,
    'Rural village areas': 9,
    'Outside rural areas': 10,
    'Hospital areas': 11,
    'School areas': 12,
    'Rural village areasOffice areas': 13  
}

df['Area_accident_occured'] = df['Area_accident_occured'].map(area)



**Valores em Falta**

In [ ]:
#df.info()

#Substituir valores em falta pela mediana
df['Driving_experience'].fillna(df['Driving_experience'].median(), inplace=True)

#Substituir valores em falta pela moda
mode_area_accident_occured = df['Area_accident_occured'].mode()[0]
df['Area_accident_occured'].fillna(mode_area_accident_occured, inplace=True)

mode_type_of_collision = df['Type_of_collision'].mode()[0]
df['Type_of_collision'].fillna(mode_type_of_collision, inplace=True)

mode_type_of_vehicle = df['Type_of_vehicle'].mode()[0]
df['Type_of_vehicle'].fillna(mode_type_of_collision, inplace=True)

#df.info()

# **PARTE 1 - DADOS NÃO BALANCEADOS**

## **Visualização dos Dados**

In [ ]:
import matplotlib.pyplot as plt

severity_counts = df['Accident_severity'].value_counts()
print(severity_counts)

plt.figure(figsize=(10, 6))
severity_counts.plot(kind='bar', color='skyblue')
plt.title('Distribuição da Gravidade dos Acidentes')
plt.xlabel('Gravidade do Acidente')
plt.ylabel('Frequência')
plt.xticks(rotation=0)  
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


In [ ]:
#Juntar as duas Injurys
df['Accident_severity2'] = df['Accident_severity'].replace({'Serious Injury': 'Serious/Fatal Injury', 'Fatal Injury': 'Serious/Fatal Injury', 'Fatal injury': 'Serious/Fatal Injury'})
#print(df['Accident_severity2'].unique())

severity_counts2 = df['Accident_severity2'].value_counts()
print(severity_counts2)

plt.figure(figsize=(10, 6))
severity_counts2.plot(kind='bar', color='skyblue')
plt.title('Distribuição da Gravidade dos Acidentes')
plt.xlabel('Gravidade do Acidente')
plt.ylabel('Frequência')
plt.xticks(rotation=0)  
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


## **Padronização e Divisão de Dados em Teste/Treino**

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#-------------------------------------------------------3 Lesões-------------------------------------------------------
X = df[features]
y = df['Accident_severity']  

#Divisão dos dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Padronização dos dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#-------------------------------------------------------2 Lesões-------------------------------------------------------

X_2 = df[features]
y_2 = df['Accident_severity2']  

#Divisão dos dados
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_2, y_2, test_size=0.2, random_state=42)

#Padronização dos dados
scaler2 = StandardScaler()
X_train_scaled2 = scaler2.fit_transform(X_train2)
X_test_scaled2 = scaler2.transform(X_test2)

## **Modelo SVM**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from scipy.stats import expon, reciprocal

modelo_svm = SVC(random_state=42)

parametros = {
    'kernel': ['linear', 'rbf'],  #dois kernels
    'C': reciprocal(1, 1000),  #distribuição log-uniforme
    'gamma': expon(scale=1.0)  #distribuição exponencial
}

#-------------------------------------------------------3 Lesões-------------------------------------------------------

search = RandomizedSearchCV(modelo_svm, parametros, n_iter=10, cv=3, scoring='f1_weighted', random_state=42, n_jobs=-1)

search.fit(X_train_scaled, y_train)

#Melhores parâmetros e melhor pontuação
print(f"Melhores parâmetros: {search.best_params_}")
print(f"Melhor pontuação F1 ponderada: {search.best_score_ * 100:.2f}%")

#Avaliação
best_model = search.best_estimator_
y_pred_SVM = best_model.predict(X_test_scaled)
print("Modelo SVM para 3 Lesões:")
print(classification_report(y_test, y_pred_SVM, zero_division=1))

#-------------------------------------------------------2 Lesões-------------------------------------------------------

search2 = RandomizedSearchCV(modelo_svm, parametros, n_iter=10, cv=3, scoring='f1_weighted', random_state=42, n_jobs=-1)

search2.fit(X_train_scaled2, y_train2)

#Melhores parâmetros e melhor pontuação
print(f"Melhores parâmetros: {search2.best_params_}")
print(f"Melhor pontuação F1 ponderada: {search2.best_score_ * 100:.2f}%")

#Avaliação
best_model = search2.best_estimator_
y_pred_SVM = best_model.predict(X_test_scaled2)
print("Modelo SVM para 2 Lesões:")
print(classification_report(y_test2, y_pred_SVM, zero_division=1))


## **Redes Neurais - Modelo Automático**

In [ ]:
#Modelo Automático

from sklearn.neural_network import MLPClassifier

#MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=1000, random_state=74)
mlp.fit(X_train_scaled, y_train)
y_pred_NN = mlp.predict(X_test_scaled)

print("Modelo Redes Neurais para 3 Lesões:")
print(classification_report(y_test, y_pred_NN, zero_division=1)) #3 Lesões

#MLPClassifier
mlp2 = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=1000, random_state=74)
mlp2.fit(X_train_scaled2, y_train2)
y_pred_NN2 = mlp2.predict(X_test_scaled2)

print("Modelo Redes Neurais para 2 Lesões:")
print(classification_report(y_test2, y_pred_NN2, zero_division=1)) #2 Lesões

## **Redes Neurais - Modelo Manual**

In [ ]:
#Modelo Manual

import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# Função de ativação sigmoid e sua derivada
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

#-------------------------------------------------------3 Lesões-------------------------------------------------------

#Codificação de classes (Valores Categóricos para Numéricos)
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

#Converter y_train_encoded para one-hot (Saída da Rede Neural)
num_classes = len(np.unique(y_train_encoded))
y_train_onehot = np.eye(num_classes)[y_train_encoded]

#Inicialização dos pesos
np.random.seed(42)
input_size = X_train_scaled.shape[1]
hidden_size = 10  
output_size = num_classes

weights1 = np.random.rand(input_size, hidden_size)
weights2 = np.random.rand(hidden_size, output_size)

#treino
learning_rate = 0.1
epochs = 10000

for epoch in range(epochs):
    #Forward propagation
    hidden_layer_input = np.dot(X_train_scaled, weights1)
    hidden_layer_output = sigmoid(hidden_layer_input)
    final_output = sigmoid(np.dot(hidden_layer_output, weights2))
    
    #Backpropagation
    error = y_train_onehot - final_output
    d_predicted_output = error * sigmoid_derivative(final_output)
    
    error_hidden_layer = d_predicted_output.dot(weights2.T)
    d_hidden_layer = error_hidden_layer * sigmoid_derivative(hidden_layer_output)
    
    #Atualização dos pesos
    weights2 += hidden_layer_output.T.dot(d_predicted_output) * learning_rate
    weights1 += X_train_scaled.T.dot(d_hidden_layer) * learning_rate

hidden_layer_input_test = np.dot(X_test_scaled, weights1)
hidden_layer_output_test = sigmoid(hidden_layer_input_test)
final_output_test = sigmoid(np.dot(hidden_layer_output_test, weights2))
y_pred_classes = np.argmax(final_output_test, axis=1)

#Converter previsões numéricas para rótulos categóricos
y_pred_labels = label_encoder.inverse_transform(y_pred_classes)

print(classification_report(y_test_encoded, y_pred_classes, target_names=label_encoder.classes_))

#-------------------------------------------------------2 Lesões-------------------------------------------------------

# Codificação de classes
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train2)
y_test_encoded = label_encoder.transform(y_test2)

#Inicialização dos pesos
np.random.seed(42)
input_size = X_train_scaled2.shape[1]
hidden_size = 10  #valor escolhido
output_size = 1  # Um neurônio de saída para classificação binária

weights1 = np.random.rand(input_size, hidden_size)
weights2 = np.random.rand(hidden_size, output_size)

#treino
learning_rate = 0.1
epochs = 10000

for epoch in range(epochs):
    #Forward propagation
    hidden_layer_input = np.dot(X_train_scaled2, weights1)
    hidden_layer_output = sigmoid(hidden_layer_input)
    final_output = sigmoid(np.dot(hidden_layer_output, weights2))
    
    #Backpropagation
    error = y_train_encoded.reshape(-1, 1) - final_output
    d_predicted_output = error * sigmoid_derivative(final_output)
    
    error_hidden_layer = d_predicted_output.dot(weights2.T)
    d_hidden_layer = error_hidden_layer * sigmoid_derivative(hidden_layer_output)
    
    #Atualização dos pesos
    weights2 += hidden_layer_output.T.dot(d_predicted_output) * learning_rate
    weights1 += X_train_scaled2.T.dot(d_hidden_layer) * learning_rate

hidden_layer_input_test = np.dot(X_test_scaled2, weights1)
hidden_layer_output_test = sigmoid(hidden_layer_input_test)
final_output_test = sigmoid(np.dot(hidden_layer_output_test, weights2))
y_pred_classes = (final_output_test > 0.5).astype(int).flatten()

y_pred_labels = label_encoder.inverse_transform(y_pred_classes)

print(classification_report(y_test_encoded, y_pred_classes, target_names=label_encoder.classes_))



## **RandomForest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

#-------------------------------------------------------3 Lesões-------------------------------------------------------

random_forest = RandomForestClassifier(n_estimators=100, random_state=42)

random_forest.fit(X_train_scaled, y_train)

y_pred_RF = random_forest.predict(X_test_scaled)

print("Modelo RandomForest para 3 Lesões:")
print(classification_report(y_test, y_pred_RF, zero_division=1))

#-------------------------------------------------------2 Lesões-------------------------------------------------------

random_forest2 = RandomForestClassifier(n_estimators=100, random_state=42)

random_forest2.fit(X_train_scaled2, y_train2)

y_pred_RF2 = random_forest2.predict(X_test_scaled2)

print("Modelo RandomForest para 2 Lesões:")
print(classification_report(y_test2, y_pred_RF2, zero_division=1))


# **PARTE 2 - DADOS BALANCEADOS**

## **Balanceamento dos Dados, Padronização e Divisão de Dados em Teste/Treino**

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import resample


#-------------------------------------------------------3 Lesões-------------------------------------------------------

X = df[features]
y = df['Accident_severity']  

#Divisão dos dados para 3 Lesões
X_train_balanced, X_test_balanced, y_train_balanced, y_test_balanced = train_test_split(X, y, test_size=0.2, random_state=42)

#-------------------------------------------------------2 Lesões-------------------------------------------------------

X_2 = df[features]
y_2 = df['Accident_severity2']  

#Divisão dos dados para 2 Lesões
X_train_balanced2, X_test_balanced2, y_train_balanced2, y_test_balanced2 = train_test_split(X_2, y_2, test_size=0.2, random_state=42)

#--------------------------------------------------Balanceamento dos dados--------------------------------------------------

#Balanceamento dos dados para 3 Lesões
train_combined = pd.concat([X_train_balanced, y_train_balanced.reset_index(drop=True)], axis=1)

slight_injury = train_combined[train_combined['Accident_severity'] == 'Slight Injury']
fatal_injury = train_combined[train_combined['Accident_severity'] == 'Fatal injury']
serious_injury = train_combined[train_combined['Accident_severity'] == 'Serious Injury']

serious_injury_upsampled = resample(serious_injury, replace=True, n_samples=len(slight_injury), random_state=42)
fatal_injury_upsampled = resample(fatal_injury, replace=True, n_samples=len(slight_injury), random_state=42)

balanced_data = pd.concat([slight_injury, serious_injury_upsampled, fatal_injury_upsampled])

X_train_balanced = balanced_data.drop('Accident_severity', axis=1)
y_train_balanced = balanced_data['Accident_severity']

#Balanceamento dos dados para 2 Lesões
train_combined2 = pd.concat([X_train_balanced2, y_train_balanced2.reset_index(drop=True)], axis=1)

slight_injury2 = train_combined2[train_combined2['Accident_severity2'] == 'Slight Injury']
serious_fatal_injury2 = train_combined2[train_combined2['Accident_severity2'] == 'Serious/Fatal Injury']

serious_fatal_injury_upsampled2 = resample(serious_fatal_injury2, replace=True, n_samples=len(slight_injury2), random_state=42)

balanced_data2 = pd.concat([slight_injury2, serious_fatal_injury_upsampled2])

X_train_balanced2 = balanced_data2.drop('Accident_severity2', axis=1)
y_train_balanced2 = balanced_data2['Accident_severity2']

#-----------------------------------------------------Padronização dos dados--------------------------------------------------

scaler = StandardScaler()

#Padronização dos dados para 3 Lesões
X_train_balanced_scaled = scaler.fit_transform(X_train_balanced)
X_test_balanced_scaled = scaler.transform(X_test_balanced)

#Padronização dos dados para 2 Lesões
X_train_balanced_scaled2 = scaler.fit_transform(X_train_balanced2)
X_test_balanced_scaled2 = scaler.transform(X_test_balanced2)


## **Visualização dos Dados**

In [ ]:
balanced_counts2 = y_train_balanced2.value_counts()
print(balanced_counts2)

plt.figure(figsize=(10, 6))
balanced_counts2.plot(kind='bar', color='skyblue')
plt.title('Distribuição da Gravidade dos Acidentes (Após Balanceamento)')
plt.xlabel('Gravidade do Acidente')
plt.ylabel('Frequência')
plt.xticks(rotation=0)  
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

balanced_counts = y_train_balanced.value_counts()
print(balanced_counts)

plt.figure(figsize=(10, 6))
balanced_counts.plot(kind='bar', color='skyblue')
plt.title('Distribuição da Gravidade dos Acidentes (Após Balanceamento)')
plt.xlabel('Gravidade do Acidente')
plt.ylabel('Frequência')
plt.xticks(rotation=0)  
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

## **Modelo SVM**

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from scipy.stats import expon, reciprocal

#preencher valores ausentes
imputer = SimpleImputer(strategy='mean')
X_train_balanced_scaled_imputed = imputer.fit_transform(X_train_balanced_scaled)
X_test_balanced_scaled_imputed = imputer.transform(X_test_balanced_scaled)

X_train_balanced_scaled2_imputed = imputer.fit_transform(X_train_balanced_scaled2)
X_test_balanced_scaled2_imputed = imputer.transform(X_test_balanced_scaled2)

#Modelo SVM
modelo_svm = SVC(random_state=42)

# Parâmetros para pesquisa aleatória
parametros = {
    'kernel': ['linear', 'rbf'],  #Dois tipos de kernel
    'C': reciprocal(1, 1000),      #Distribuição log-uniforme
    'gamma': expon(scale=1.0)      #Distribuição exponencial
}

#-------------------------------------------------------3 Lesões-------------------------------------------------------

search = RandomizedSearchCV(modelo_svm, parametros, n_iter=10, cv=3, scoring='f1_weighted', random_state=42, n_jobs=-1)
search.fit(X_train_balanced_scaled_imputed, y_train_balanced)

print(f"Melhores parâmetros para 3 Lesões: {search.best_params_}")
print(f"Melhor pontuação F1 ponderada para 3 Lesões: {search.best_score_ * 100:.2f}%")

y_pred_SVM = search.best_estimator_.predict(X_test_balanced_scaled_imputed)
print("\nModelo SVM para 3 Lesões:")
print(classification_report(y_test_balanced, y_pred_SVM, zero_division=1))

#-------------------------------------------------------2 Lesões-------------------------------------------------------

search2 = RandomizedSearchCV(modelo_svm, parametros, n_iter=10, cv=3, scoring='f1_weighted', random_state=42, n_jobs=-1)
search2.fit(X_train_balanced_scaled2_imputed, y_train_balanced2)

print(f"\nMelhores parâmetros para 2 Lesões: {search2.best_params_}")
print(f"Melhor pontuação F1 ponderada para 2 Lesões: {search2.best_score_ * 100:.2f}%")

y_pred_SVM2 = search2.best_estimator_.predict(X_test_balanced_scaled2_imputed)
print("\nModelo SVM para 2 Lesões:")
print(classification_report(y_test_balanced2, y_pred_SVM2, zero_division=1))


## **Redes Neurais - Modelo Automático**

In [ ]:
from sklearn.impute import SimpleImputer

mlp_balanced = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=1000, random_state=74)

imputer = SimpleImputer(strategy='mean')
X_train_balanced_scaled_imputed = imputer.fit_transform(X_train_balanced_scaled)
X_test_balanced_scaled_imputed = imputer.transform(X_test_balanced_scaled)

mlp_balanced.fit(X_train_balanced_scaled_imputed, y_train_balanced)
y_pred_NN_balanced = mlp_balanced.predict(X_test_balanced_scaled_imputed)

print("Modelo Redes Neurais para 3 Lesões (Dados Balanceados):")
print(classification_report(y_test_balanced, y_pred_NN_balanced, zero_division=1))#3 Lesões

mlp_balanced2 = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=1000, random_state=74)

#substituir valores ausentes com a média
imputer = SimpleImputer(strategy='mean')
X_train_balanced_scaled2_imputed = imputer.fit_transform(X_train_balanced_scaled2)
X_test_balanced_scaled2_imputed = imputer.transform(X_test_balanced_scaled2)

mlp_balanced2.fit(X_train_balanced_scaled2_imputed, y_train_balanced2)
y_pred_NN_balanced2 = mlp_balanced2.predict(X_test_balanced_scaled2_imputed)

print("Modelo Redes Neurais para 2 Lesões (Dados Balanceados):")
print(classification_report(y_test_balanced2, y_pred_NN_balanced2, zero_division=1))#2 Lesões


## **Redes Neurais - Modelo Manual**

In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

#Função de ativação sigmoid e sua derivada
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

#-------------------------------------------------------3 Lesões com dados balanceados-------------------------------------------------------

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train_balanced)
y_test_encoded = label_encoder.transform(y_test_balanced)

#Converter y_train_encoded para one-hot
num_classes = len(np.unique(y_train_encoded))
y_train_onehot = np.eye(num_classes)[y_train_encoded]

np.random.seed(42)
input_size = X_train_balanced_scaled.shape[1]
hidden_size = 10  
output_size = num_classes

weights1 = np.random.rand(input_size, hidden_size)
weights2 = np.random.rand(hidden_size, output_size)

learning_rate = 0.1
epochs = 10000

for epoch in range(epochs):
    hidden_layer_input = np.dot(X_train_balanced_scaled, weights1)
    hidden_layer_output = sigmoid(hidden_layer_input)
    final_output = sigmoid(np.dot(hidden_layer_output, weights2))
    
    error = y_train_onehot - final_output
    d_predicted_output = error * sigmoid_derivative(final_output)
    
    error_hidden_layer = d_predicted_output.dot(weights2.T)
    d_hidden_layer = error_hidden_layer * sigmoid_derivative(hidden_layer_output)
    
    weights2 += hidden_layer_output.T.dot(d_predicted_output) * learning_rate
    weights1 += X_train_balanced_scaled.T.dot(d_hidden_layer) * learning_rate

#dados de teste
hidden_layer_input_test = np.dot(X_test_balanced_scaled, weights1)
hidden_layer_output_test = sigmoid(hidden_layer_input_test)
final_output_test = sigmoid(np.dot(hidden_layer_output_test, weights2))
y_pred_classes = np.argmax(final_output_test, axis=1)
y_pred_labels = label_encoder.inverse_transform(y_pred_classes)
print(classification_report(y_test_encoded, y_pred_classes, target_names=label_encoder.classes_))

#-------------------------------------------------------2 Lesões com dados balanceados-------------------------------------------------------

#codificação de classes
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train_balanced2)
y_test_encoded = label_encoder.transform(y_test_balanced2)

np.random.seed(42)
input_size = X_train_balanced_scaled2.shape[1]
hidden_size = 10  #valor escolhido
output_size = 1  #Um neurônio de saída para classificação binária

weights1 = np.random.rand(input_size, hidden_size)
weights2 = np.random.rand(hidden_size, output_size)

#treino
learning_rate = 0.1
epochs = 10000

for epoch in range(epochs):
    #Forward propagation
    hidden_layer_input = np.dot(X_train_balanced_scaled2, weights1)
    hidden_layer_output = sigmoid(hidden_layer_input)
    final_output = sigmoid(np.dot(hidden_layer_output, weights2))
    
    #Backpropagation
    error = y_train_encoded.reshape(-1, 1) - final_output
    d_predicted_output = error * sigmoid_derivative(final_output)
    
    error_hidden_layer = d_predicted_output.dot(weights2.T)
    d_hidden_layer = error_hidden_layer * sigmoid_derivative(hidden_layer_output)
    
    #Atualização dos pesos
    weights2 += hidden_layer_output.T.dot(d_predicted_output) * learning_rate
    weights1 += X_train_balanced_scaled2.T.dot(d_hidden_layer) * learning_rate

#dados de teste
hidden_layer_input_test = np.dot(X_test_balanced_scaled2, weights1)
hidden_layer_output_test = sigmoid(hidden_layer_input_test)
final_output_test = sigmoid(np.dot(hidden_layer_output_test, weights2))
y_pred_classes = (final_output_test > 0.5).astype(int).flatten()
y_pred_labels = label_encoder.inverse_transform(y_pred_classes)
print(classification_report(y_test_encoded, y_pred_classes, target_names=label_encoder.classes_))


## **RandomForest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')

#preencher valores ausentes nos dados de treinamento e teste
X_train_balanced_scaled_imputed = imputer.fit_transform(X_train_balanced_scaled)
X_test_balanced_scaled_imputed = imputer.transform(X_test_balanced_scaled)

X_train_balanced_scaled2_imputed = imputer.fit_transform(X_train_balanced_scaled2)
X_test_balanced_scaled2_imputed = imputer.transform(X_test_balanced_scaled2)

#-------------------------------------------------------3 Lesões-------------------------------------------------------

random_forest_balanced = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_balanced.fit(X_train_balanced_scaled_imputed, y_train_balanced)

y_pred_RF_balanced = random_forest_balanced.predict(X_test_balanced_scaled_imputed)

print("Modelo RandomForest para 3 Lesões (Dados Balanceados):")
print(classification_report(y_test_balanced, y_pred_RF_balanced, zero_division=1))

#-------------------------------------------------------2 Lesões-------------------------------------------------------

random_forest_balanced2 = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_balanced2.fit(X_train_balanced_scaled2_imputed, y_train_balanced2)

y_pred_RF_balanced2 = random_forest_balanced2.predict(X_test_balanced_scaled2_imputed)

print("Modelo RandomForest para 2 Lesões (Dados Balanceados):")
print(classification_report(y_test_balanced2, y_pred_RF_balanced2, zero_division=1))
